# 검수하는 자동화 코드 작성

In [6]:
import pandas as pd
import re

In [4]:
df = pd.read_csv("Curriculum_Mid.csv")
sample_df = df[:10]
sample_df.head()

,textbook ID,학년,출판사,교육과정,Lesson,구분,Title,DESCRIPTION,Key Expression (영어),Key Expression (한국어),...,Index 1,Quiz 2,Korean 2,Option 2-1,Option 2-2,Option 2-3,Answer 2,Index 2,샘플 스크립트 (영어),샘플 스크립트 (한국어)
0,701,중1,능률(김),2015,1,function,Welcome to My World,"자기소개와 출신지에 대해 말하고, 이에 반응하기.",A: My name is Felix. I’m from Australia.\n B: ...,A: 내 이름은 필릭스야. 나는 호주에서 왔어.\n B: 만나서 반가워.,...,2,"A: Hi, I’m Sam. I ____ from Korea. B: Nice to ...","A: 안녕, 나는 Sam이야. 나는 Korea에서 왔어. B: 만나서 반가워.",live,come,am,come,2,A: My name is Leo. I’m from Canada. B: Pleased...,A: 내 이름은 Leo야. 나는 Canada 출신이야. B: 만나서 반가워. A: ...
1,701,중1,능률(김),2015,1,function,Welcome to My World,like와 favorite 단어를 활용해 좋아하는 것에 대해 묻고 답하기.,A: What is your favorite sport?\n B: I like so...,A: 네가 가장 좋아하는 스포츠는 뭐야?\n B: 나는 축구를 좋아해.,...,2,What do you ______ the most?,가장 좋아하는 것은 무엇이에요?,prefer,love,like,like,3,A: What's your favorite game? B: I like basket...,A: 네가 가장 좋아하는 게임은 무엇이에요? B: 저는 농구를 좋아해요. A: 주말...
2,701,중1,능률(김),2015,2,function,Discover Your Culture,be going to ~를 활용해 계획이나 예정된 일에 대해 묻고 답하기.,A: What are you going to do on your holiday? B...,A: 너는 휴가 때 무엇을 할 예정이야? B: 나는 여행을 갈 예정이야.,...,2,I'm _____ to try a new recipe.,나는 새로운 요리를 시도할 계획이야.,planning,preparing,deciding,planning,1,A: What are you going to do this weekend? B: I...,A: 이번 주말에 무엇을 할 거야? B: 하이킹을 갈 거야. A: 저녁에 어떤 계획...
3,701,중1,능률(김),2015,2,function,Discover Your Culture,There is/are ~을 활용하여 장소 설명하기.,There are many parks in my city.,내 도시에 공원이 많이 있어.,...,2,You can _____ many cafes.,많은 카페를 찾을 수 있어요.,see,find,buy,find,2,A: There are many libraries in my town. B: Rea...,A: 우리 동네에는 많은 도서관이 있어요. B: 정말요? 그곳에서 시간을 보내는 걸...
4,701,중1,능률(김),2015,3,function,"Spend Smart, Save Smart",You should ~를 활용해 조언하기.,"A: You should study harder.\n B: Okay, I’ll try.","A: 너는 더 열심히 공부해야 해.\n B: 알겠어, 노력할게.",...,1,A: It’s a good idea to ______. B: I’ll give it...,A: ~하는 것이 좋은 생각이야. B: 한 번 해볼게.,run fast,study harder,eat quickly,study harder,2,"A: You should exercise regularly. B: Okay, I'l...","A: 너는 규칙적으로 운동해야 해. B: 알겠어, 노력해볼게. A: 균형 잡힌 식단..."


In [5]:
df.columns

Index(['textbook ID', '학년', '출판사', '교육과정', 'Lesson', '구분', 'Title',
       'DESCRIPTION', 'Key Expression (영어)', 'Key Expression (한국어)',
       'Sub Expression (영어)', 'Sub Expression (한국어)', '학습목표', 'Quiz 1',
       'Korean 1', 'Option 1-1', 'Option 1-2', 'Option 1-3', 'Answer 1',
       'Index 1', 'Quiz 2', 'Korean 2', 'Option 2-1', 'Option 2-2',
       'Option 2-3', 'Answer 2', 'Index 2', '샘플 스크립트 (영어)', '샘플 스크립트 (한국어)'],
      dtype='object')

In [ ]:
def check_valid_fill_in_blank_quiz(self, quiz: dict) -> bool:
    fill_in_blank_quiz = quiz.get("fill_in_blank_quiz")
    answer = quiz.get("answer")
    options = quiz.get("options")
    key_expression = quiz.get("key_expression")
    answer_index = quiz.get("answer_index")
    # 0. 필드값 누락 체크
    if (
        not fill_in_blank_quiz
        or not answer
        or not options
        or not answer_index
        or not key_expression
    ):
        return False

    # 1. 빈칸이 정확히 하나, 다섯개의 언더바로 구성되어있는지 체크
    if len(re.findall(self.appropriate_blank, fill_in_blank_quiz)) != 1:
        return False
    # 2. 정답이 옵션에 포함되어 있는지 체크
    if answer not in options:
        return False
    if isinstance(answer_index, str) and not answer_index.isnumeric():
        return False
    # 3. 정답의 인덱스가 answer_index와 일치하는지 체크
    answer_index = int(answer_index)
    if options[answer_index - 1] != answer:
        return False
    # 4. blank 자리에 정답을 넣었을 때, key expression과 일치하는지 체크
    filled_sentence = fill_in_blank_quiz.replace(
        str(self.appropriate_blank), answer
    )
    if filled_sentence.lower() != key_expression.lower():
        return False
    return True